In [1]:
!pip install scikit-learn=='0.23.1'

     |████████████████████████████████| 6.8MB 8.3MB/s 
  Found existing installation: scikit-learn 0.22.2.post1
    Uninstalling scikit-learn-0.22.2.post1:
      Successfully uninstalled scikit-learn-0.22.2.post1


In [16]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.model_selection import *
from sklearn.metrics import *
from sklearn.ensemble import *
from sklearn.preprocessing import StandardScaler


pd.set_option('max_column', 10000)

import warnings
warnings.filterwarnings('ignore')

In [3]:
train = pd.read_csv('/content/drive/MyDrive/Silicon/Train.csv')
test = pd.read_csv('/content/drive/MyDrive/Silicon/Test.csv')
sub = pd.read_csv('/content/drive/MyDrive/Silicon/SampleSubmission.csv')

In [4]:
train.shape, test.shape, sub.shape

((14016, 17), (1168, 16), (1168, 2))

In [5]:
len_train = len(train)
df = pd.concat([train, test]).reset_index(drop=True)

In [6]:
df['sex'] = df['sex'].replace({'MF':'M'})

In [7]:
def create_bin_features(input_df, features):
    for bin_fe in features:
        print("Binning: ",bin_fe)
        input_df[bin_fe+"_BINS"] = pd.qcut(input_df[bin_fe], 4, labels=False, duplicates='drop')
    
    return input_df

In [8]:
binning_num_features = ['percentage_no_schooling','literate_pop']

df = create_bin_features(df, binning_num_features)

Binning:  percentage_no_schooling
Binning:  literate_pop


In [9]:
df['pop_no_school'] = df['percentage_no_schooling']*df['literate_pop'] 

In [10]:
colss = ['percentage_no_schooling', 'percentage_primary_schooling',
       'percentage_primary_schooling_completed',
       'percentage_secondary_schooling',
       'percentage_secondary_schooling_completed',
       'percentage_tertiary_schooling',
       'percentage_tertiary_schooling_completed','target','yr_sch','pop_no_school']

df[colss] = df[colss].round(2)

In [11]:
df = pd.get_dummies(df, columns=['region_code'])

In [12]:
train = df[:len_train]
test = df[len_train:]

In [13]:
X = train.drop(columns=['ID','target', 'ageto','countryCode','year','sex'])
y = train['target']
tes = test.drop(columns=['ID','target', 'ageto','countryCode','year','sex'])

In [14]:
ss = StandardScaler()
X = ss.fit_transform(X)
tes = ss.transform(tes)

In [17]:
%%time
test_store=[]

oofs = np.zeros((len(train)))
y_oof = np.zeros((len(train)))
preds = np.zeros((len(test)))


fold = KFold(n_splits=12, shuffle=True, random_state=212)

i=1
for train_index, test_index in fold.split(X, pd.qcut(y, 10, labels=False, duplicates='drop')):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    
    y_oof[test_index] = y_test
    
    lgb = RandomForestRegressor(n_estimators=500, random_state=42)
    lgb.fit(X_train,y_train)
    predict = lgb.predict(X_test)
    
    oofs[test_index] = predict

    pred = lgb.predict(tes)
    test_store.append(pred)
    
    preds += pred/12

CPU times: user 11min 50s, sys: 1.62 s, total: 11min 52s
Wall time: 11min 50s


In [18]:
sub['target'] = preds

In [19]:
sub.head()

,ID,target
0,ID_00ILO0W,87.486918
1,ID_00KPO3Y,83.217815
2,ID_00MLY2L,88.368663
3,ID_00PVY5K,95.207203
4,ID_00TLQ4J,97.823895


In [20]:
rf = pd.DataFrame(oofs, columns=['rf'])
rf_t = pd.DataFrame(preds, columns=['rf'])

rf.to_csv('rf_oof.csv', index=False)
rf_t.to_csv('rf_pred.csv', index=False)